In [5]:
import import_ipynb
import abuse_detecting_preprocessing as pp#text_preprocessing(x) ,text_preprocessing_can_eng_num(x)
import pandas as pd
import numpy as np
from tqdm import tqdm
import fasttext
from glob import glob
import os
import random

importing Jupyter notebook from abuse_detecting_preprocessing.ipynb
['ㅇㅡㄴㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'ㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 

['t1ㅇㅡㄴlckㄹㅡㄹㅈㅏㄹㅇㅣ-ㄱㅣㄴㄷㅏ-ㅎㅏ-', 'fakerㄴㅡㄴㄹㅗㄹㅇㅢ-ㅅㅣㄴㅇㅣ-ㄷㅏ-'] 



/opt/homebrew/anaconda3/envs/yongari/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def erase_bot(data):

    # 첫 번째 열의 값이 '@'로 시작하는 행을 필터링하여 봇으로 간주하고 제거
    idxs = data[data.str.startswith('@') == True].index
    data.loc[idxs] = None
    bot_cnt = len(idxs)
    print('data.shape : ', data.shape, 'None 처리한 봇 수 : ', bot_cnt, '\n')
    return data
# erase_bot 함수 사용 예시
# data = erase_bot(data)


In [7]:
def null_finder(data): # 공백만 있는 메세지 제거
    print('공백 제거 전\n', data.isnull().value_counts())
    data = data.dropna()
    print('공백 제거 후 \n', data.isnull().value_counts())
    
    return data

In [8]:
def erase_emoticons(data):
    # 각 문장에 대해 이모티콘을 제거하는 작업을 수행
    for i in range(len(data)):
        while '::' in data[i]:
            start = data[i].find('::')
            end = data[i].find('::', start + 2)
            if end == -1:
                break
            data[i] = data[i][:start] + data[i][end + 2:]
    return data


In [9]:
def equalmatchfinder(routes):
    routes = sorted(routes)
    team_l = {'T1' : 0, 'GEN' : 0, 'HLE' : 0, 'KT' : 0, 'KDF' : 0,
              'DK' : 0, 'FOX' : 0, 'NS' : 0, 'DRX' : 0, 'BRO' : 0}
    idx = 0
    for route in routes:
        for key in team_l.keys():
            if route.find(key) != -1:
                team_l[key] += 1
        idx += 1
        if len(set(list(team_l.values()))) == 1: # set으로 바꾸면 중복이 사라지기에 길이가 1이라는 뜻은 경기수가 전부 같음을 의미.
            print(idx,'번째에서 모두 같은 값입니다.\n', team_l)
    
    # 1~10 : fasttext 
    # 11~15 : activation learning
    # 16~20 : analysis
    # 21~25 : labeling

In [10]:
def n_grams(sentences, n):
    n_grams_list = []

    for sentence in sentences:
        n_grams_sentence = []
        sentence_length = len(sentence)

        if sentence_length < n:
            continue

        for i in range(sentence_length - n + 1):
            n_gram = sentence[i:i+n]
            n_grams_sentence.append(n_gram)

        n_grams_list.append(n_grams_sentence)
    print(f' n = {n}인 ngram 생성.')
    return n_grams_list

In [11]:
def fasttext_model(x,n):

    with open('tokenized_data.txt', 'w') as out:
        for line in tqdm(x, unit=' line'):
            out.write(' '.join(line) + '\n')
            
    model = fasttext.train_unsupervised('tokenized_data.txt',
                                    model = 'skipgram', lr = 0.05, dim = 100,
                                    ws = 5, epoch = 30, minn = 1, word_ngrams = n)
    return model

In [12]:
def making_fasttext_model(messages, N = 5):
    messages = erase_bot(messages)

    messages = null_finder(messages)

    messages = messages.reset_index(drop = True)
    messages = erase_emoticons(messages)
    
    words = pp.text_preprocessing_can_eng_num(messages)
        # @로 시작하는 채팅들을 Null로 설정 (봇 제거)
        # Null들을 제거
        # :로 시작하는 이모티콘들 제거
        # 메세지에서 한국어와 영어만 남긴다.
        # 텍스트에서 겹친 글자를 줄인다. (ㅋㅋㅋㅋㅋ > ㅋ)
        # 텍스트를 초성으로 바꾼다.
        # 바꾼 텍스트에서 겹모음을 바꾼다.
    bow = n_grams(words , N) # 리스트로 받아 각 문장을 5개씩 쪼갠다.
    model = fasttext_model(bow, N)
    return model


In [13]:
route = 'chatting_datas/*.csv'
routes = glob(route)
routes = sorted(routes)


In [14]:
equalmatchfinder(routes)
len(routes)
# 25경기까지는 모두 동일한 횟수로 경기하였음.

5 번째에서 모두 같은 값입니다.
 {'T1': 2, 'GEN': 2, 'HLE': 2, 'KT': 2, 'KDF': 2, 'DK': 2, 'FOX': 2, 'NS': 2, 'DRX': 2, 'BRO': 2}
10 번째에서 모두 같은 값입니다.
 {'T1': 4, 'GEN': 4, 'HLE': 4, 'KT': 4, 'KDF': 4, 'DK': 4, 'FOX': 4, 'NS': 4, 'DRX': 4, 'BRO': 4}
15 번째에서 모두 같은 값입니다.
 {'T1': 6, 'GEN': 6, 'HLE': 6, 'KT': 6, 'KDF': 6, 'DK': 6, 'FOX': 6, 'NS': 6, 'DRX': 6, 'BRO': 6}
20 번째에서 모두 같은 값입니다.
 {'T1': 8, 'GEN': 8, 'HLE': 8, 'KT': 8, 'KDF': 8, 'DK': 8, 'FOX': 8, 'NS': 8, 'DRX': 8, 'BRO': 8}
26 번째에서 모두 같은 값입니다.
 {'T1': 10, 'GEN': 10, 'HLE': 10, 'KT': 10, 'KDF': 10, 'DK': 10, 'FOX': 10, 'NS': 10, 'DRX': 10, 'BRO': 10}


57

In [15]:
import gc
Games = 26

df = pd.DataFrame()
for r in routes[:Games]:
    data = pd.read_csv(r)
    df = pd.concat([df, data])
    del data
    gc.collect()
df = df[['message']].reset_index(drop = True)
print(df.shape)
df.sample(5)


(482915, 1)


,message
174460,타워있는데 넥서스 칠수있나?
269144,대상혁
208482,스찌.?
179624,가슴이 옹졸해 지는 경기 보러 왔따.
111739,퍼펙트 못한다고 하기엔 킹겐 솔킬냈쥬?


In [ ]:
model = making_fasttext_model(df['message'], 5)

data.shape :  (482915,) None 처리한 봇 수 :  4054 

공백 제거 전
 message
False    478850
True       4065
Name: count, dtype: int64
공백 제거 후 
 message
False    478850
Name: count, dtype: int64
 n = 5인 ngram 생성.


100%|████████████████████████████| 430038/430038 [00:00<00:00, 838779.27 line/s]
Read 11M words
Number of words:  115934
Number of labels: 0
Progress:   1.7% words/sec/thread:   53677 lr:  0.049161 avg.loss:  0.863549 ETA:   0h14m54s

In [ ]:
model.get_nearest_neighbors('ㅍㅔ-ㅇㅣ')

In [45]:
model.save_model('fasttext_model.bin')